### Задание 1
Вы работает junior аналитиком в компании, создающей мобильные игры. Вам необходимо проанализировать данные, содержащие информацию об игроке, - его уровень, страна, платформа, дата регистрации и дата login’a. Эти данные помогут сформулировать представление об аудитории игры. 
Таблица содержит следующие столбцы:
*       **country** - страна,  
*       **pl** - уровень игрока,
*       **plf** - платформа, 
*       **reg** - дата регистрации игрока, 
*       **ts** - время последнего захода игрока в игру, 
*       **user** - id игрока.


### Вопросы
1. Для каждой платформы определите топ 5 стран по количеству игроков? 
2. Какой максимальный lifetime у игроков в игре?
3. Какое количество игроков находится выше 20 уровня? 
4. Посчитайте средний уровень игроков для каждой страны?


Импорт нужных библиотек, выгрузка датасета и проверка датасета на пропущенные значения

In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv('./analytics_test.csv')
df.isna().sum() #проверка датасета на пустые ячейки, и отсутствующие ячейки.

country    0
pl         0
plf        0
reg        0
ts         0
user       0
dtype: int64

Пропущенных данных нет


In [9]:
platforms = df['plf'].unique() #Формирование массива из уникальных платформ

#### Вопрос 1

In [10]:
task_1 = df.groupby(['plf', 'country']).count()['user'].sort_values(ascending=False) #группировка по платформе и стране для
print(task_1)                                                                        #формирования топ 5 стран для каждой 
                                                                                     #платформы по количеству игроков

plf      country
Android  IN         151
iOS      CN         122
Android  RU          59
         CN          58
         ID          36
                   ... 
         NZ           1
         NP           1
         NI           1
         MR           1
         QA           1
Name: user, Length: 153, dtype: int64


In [11]:
for platform in platforms:
    print(f'Платформа: {platform}')
    print(f'Топ 5 стран по количеству игроков на этой платформе: {task_1[platform].head(5)}\n') #Вывод платформы и топ 5 стран
                                                                                                #по количеству игроков

Платформа: Android
Топ 5 стран по количеству игроков на этой платформе: country
IN    151
RU     59
CN     58
ID     36
PH     30
Name: user, dtype: int64

Платформа: iOS
Топ 5 стран по количеству игроков на этой платформе: country
CN    122
VN     18
US     14
RU     12
IN     11
Name: user, dtype: int64

Платформа: Samsung
Топ 5 стран по количеству игроков на этой платформе: country
EG    3
US    1
TN    1
NG    1
IN    1
Name: user, dtype: int64

Платформа: Amazon
Топ 5 стран по количеству игроков на этой платформе: country
CI    1
US    1
Name: user, dtype: int64



##### Вывод: Подавляющее число игроков использует такие платформы, как Android и iOS. Самой мало используемой платформой является Amazon. 

#### Вопрос 2

В ходе выполнения этого вопроса, было выявлено, что данные о дате регистрации и времени последнего захода в игру представлены в
формате Unix-timestamp

In [12]:
from datetime import timedelta

df['reg_time'] = pd.to_datetime(df['reg'], unit='ms', origin='unix') #Преобразование даты регистрации в
                                                                     #стандартный формат даты и времени
df['ts_time'] = pd.to_datetime(df['ts'], unit='ms', origin='unix') #Преобразование времени последнего захода в игру
                                                                   #в стандартный формат даты и времени

df.head()

,country,pl,plf,reg,ts,user,reg_time,ts_time
0,IT,33,Android,1511191920859,1586044805457,1122239prod-eu,2017-11-20 15:32:00.859,2020-04-05 00:00:05.457
1,IN,33,Android,1509074590554,1586045807421,147065prod-us,2017-10-27 03:23:10.554,2020-04-05 00:16:47.421
2,TR,33,iOS,1575420775882,1586047059117,25564086prod-eu,2019-12-04 00:52:55.882,2020-04-05 00:37:39.117
3,HK,12,Android,1584406076183,1586045087568,10123050prod-as,2020-03-17 00:47:56.183,2020-04-05 00:04:47.568
4,CN,21,Android,1581887511291,1586046421744,9893956prod-as,2020-02-16 21:11:51.291,2020-04-05 00:27:01.744


In [40]:
df.loc[:, 'lifetime'] = df['ts_time']-df['reg_time']
df.loc[:, 'lifetime'] = df['lifetime'] / timedelta(days=1)
df.loc[:, 'lifetime'] = (df['lifetime'] // 1) + 1
df['lifetime'].max()

891.0

##### Вывод: Максимальный lifetime у игроков в игре это примерно 891 день. Это скорее всего самые старые игроки в игре. Также присутствуют новые игроки, которые буквально только что зарегистрировались в игре.

#### Вопрос 3

In [50]:
df['pl'].max()

33

In [48]:
df[df['pl'] > 20].count()['user'] #Посчет количества игроков у которых уровень больше 20

250

##### Вывод: Четверть игроков имеет уровень больше 20. Я считаю, что уже можно задуматься о повышении максимального уровня. На данный момент, он ограничен 33 уровнем.

#### Вопрос 4

In [55]:
task_4 = df.groupby(['country']).apply(np.mean)['pl'] #Группировка по стране с применением функции apply для вычисления среднего
                                                      #значения уровня для каждой страны в датасете
    
print(task_4)

country
AE    12.833333
AF    11.666667
AL    12.000000
AM    11.666667
AO    22.000000
        ...    
VN    17.000000
XK    24.000000
YE    30.000000
ZA     6.000000
ZM    33.000000
Name: pl, Length: 102, dtype: float64


In [59]:
task_4.describe(include = [np.number])

count    102.000000
mean      13.265186
std        7.644170
min        1.000000
25%        6.708333
50%       12.621693
75%       17.000000
max       33.000000
Name: pl, dtype: float64

In [60]:
task_4[task_4 == 33.0]

country
AT    33.0
QA    33.0
ZM    33.0
Name: pl, dtype: float64

In [61]:
task_4[task_4 == 1.0]

country
SN    1.0
Name: pl, dtype: float64

##### Вывод: Можно выделить 3 страны, которые имеют средний уровень игроков по стране 33. Это может говорить о малом числе игроков в этой стране. Тоже самое можно сказать, про игроков из страны со средним уровнем 1. Также можно сказать, что средний уровень игроков по миру равен 13.26.

### Задание 2
Вы попали на аналитическую конференцию. В ней приняли участие 80 специалистов: 40 продуктовых аналитиков и 40 ML-инженеров. 30 продуктовых аналитиков и 20 ML-инженеров пойдут на лекцию, посвященную архитектуре баз данных. Известно, что случайно выбранный продуктовый аналитик пойдет на лекцию по архитектуре баз данных с вероятностью 75%. Какова вероятность того, что случайно выбранный посетитель лекции, посвященной архитектуре баз данных, окажется ML-инженером? 


**Ответ**: 
Всего на лекцию посвященную базам данных пойдут 50 человек, 30 продуктовых аналитиков и 20 ML-инженеров. Вероятность того, что случайно выбранный посетитель лекции окажется ML-инженером, будет равна 20/50. Т.е. 20 ML-инженеров разделить на общее количество посетителей лекции. Вероятность равна 0.4 или 40%.

### Задание 3
Есть поверье, что если спортсмен побывал на обложке  “Sports illustrated”, то вскоре после этого с этим спортсменом случается спортивная неудача или снижение спортивного результата. Одной жертвой “проклятья”  в 2000 году стал русский гимнаст Иван Иванков, которого назвали лучшим спортсменом мира, после чего он не смог даже отобраться на летние Олимпийские игры.
Действительно “Sports illustrated” является носителем проклятья или это можно объяснить каким-либо феноменом?

**Ответ**: 
Я считаю, что никакого проклятия на этом спортивном журнале не лежит. Скорее всего, однажды, произошел следующий случай. Спортсмен, который был изображен на обложке журнала, сразу после выпуска потерпел спортивную неудачу, которая не была связана с журналом, просто случайность. Люди заметили связь этих двух событий, выпуска журнала и спортивной неудачи, и стали распростронять слухи, что журнал проклят. Последующие спортивные неудачи спортсменов, которые также попадали на обложку этого журнала, можно объяснить самовнушением спортсменов. Они, после попадания на обложку, вспоминали поверье, и начинали терять уверенность в себе, что и приводило их к неудачам.  

### Задание 4
Обязательно приложите сопроводительное письмо, в котором опишите как часто играете в игры, какие ваши любимые игры и что вам в них нравится, как вы относитесь к мобильным играм? 


В игры я играю, каждый вечер, где-то по часу. Среди моих любимых игр, могу выделить Genshin Impact, Diablo 3, Heroes of Might and Magic 3. В Genshin Impact мне нравиться система прокачки персонажей, улучшения оружия, случайная составляющая при "выбивании" новых персонажей. В Diablo 3 мне нравиться случайная составляющая при получении различных предметов и возможность состязаться с другими игроками. В HoMM3 мне нравиться тактика и стратегия для победы над противником. К мобильным играм отношусь хорошо, но предпочитаю играть на ПК.